<a href="https://colab.research.google.com/github/Aksh1312/simple-registration-form/blob/main/Terrain_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
!unzip /content/EuroSAT_RGB.zip -d /content/dataset

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2

# Define ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2  # Split for validation
)

train_generator = datagen.flow_from_directory(
    '/content/dataset/EuroSAT_RGB',
    target_size=(256, 256),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    '/content/dataset/EuroSAT_RGB',
    target_size=(256, 256),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(256, 256, 3)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')  # Number of classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=3,
    validation_data=validation_generator
)

# Save the model
model.save('/content/land_use_classifier_model.h5')

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# Define class names manually
class_names = ['Desert', 'Forest Cover', 'Mountains', 'Urban', 'Water bodies']

# Function to classify new images
def classify_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found or unable to load: {image_path}")
    image = cv2.resize(image, (256, 256))
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)

    predictions = model.predict(image)
    class_index = np.argmax(predictions, axis=1)[0]
    return class_index

# Example usage
new_image_path = '/content/dataset/EuroSAT_RGB/Pasture/Pasture_130.jpg'
class_index = classify_image(new_image_path)
print(f'Predicted class: {class_names[class_index]}')